###Ingestion del archivo "movie_cast.json"

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

####Paso 1- Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
movie_cast_schema = StructType(fields=[
    StructField("movieId", IntegerType(), True),
    StructField("personId", IntegerType(), True),
    StructField("characterName", StringType(), True),
    StructField("genderId", IntegerType(), True),
    StructField("castOrder", IntegerType(), True)
])

In [0]:
movie_cast_df = spark.read \
                .schema(movie_cast_schema) \
                .option("multiLine", True) \
                .json(f"{bronze_folder_path}/{v_file_date}/movie_cast.json")

In [0]:
display(movie_cast_df)

####Paso 2 - Renombrar las columnas y añadir nuevas columnas
- "movieId" renombar a "movie_id", "personId" a "person_id" y "characterName" a "character_name"
- Agregar las columnas "ingestion_date" y "environment"

In [0]:
from pyspark.sql.functions import col, current_timestamp, lit

In [0]:
movie_cast_with_columns_df = add_ingestion_dat(movie_cast_df) \
                            .withColumnsRenamed({"movieId":"movie_id",
                                                  "personId":"person_id",
                                                  "characterName":"character_name"}) \
                            .withColumn("environment", lit(v_environment)) \
                            .withColumn("file_date", lit(v_file_date))
display(movie_cast_with_columns_df)

####Paso 3 - Eliminar las columnas no deseadas del DataFrame

In [0]:
from pyspark.sql.functions import col

In [0]:
movie_cast_final_df = movie_cast_with_columns_df.drop(col("genderId"), col("castOrder"))
display(movie_cast_final_df)

####Paso 4 - Escribir la salida en un formato "Parquet"


In [0]:
#movie_cast_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/movies_cast")
#movie_cast_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.movies_cast")
merge_condition='tgt.movie_id = src.movie_id and tgt.person_id = src.person_id and tgt.file_date = src.file_date'
merge_delta_lake(movie_cast_final_df, "movie_silver", "movies_cast", silver_folder_path, merge_condition, "file_date")

In [0]:
%sql
select file_date, count(1) 
from movie_silver.movies_cast
group by file_date;

In [0]:
display(spark.read.format("delta").load("/mnt/mymoviehistory/silver/movies_cast"))

In [0]:
dbutils.notebook.exit("Success")